In [1]:
import numpy as np
import matplotlib.pyplot as plt  # To visualize
import pandas as pd  # To read data
from sklearn.linear_model import LinearRegression

In [2]:
def normalize_list(data):
	x_min = min(data)
	x_max = max(data)
	if x_max-x_min == 0:
		return None
	return [(x-x_min)/(x_max-x_min) for x in data]

In [3]:
df = pd.read_csv('../../data_with_senti/ge_sentiments.csv', encoding='utf-8')

In [4]:
df.head()

,Unnamed: 0,Company,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Review_Title_sentiment_score_vader,Pros_And_Cons,Pros_And_Cons_sentiment_score_vader
0,0,GE,5.0,0,NaN,Good company,2022-02-25,Technical Product Manager,"Detroit, MI",2,0,1,Competitive salary and training opportunities,Extreme work load and unclear responsibility.,NaN,0.44,Competitive salary and training opportunitiesE...,-0.08
1,1,GE,4.0,0,NaN,Its electric,2022-02-24,Anonymous Employee,NaN,0,0,0,"lots of power, lots of light","to bright sometimes, less power",NaN,0.00,"lots of power, lots of lightto bright sometime...",0.44
2,2,GE,5.0,1,less than 1 year,Good Company to start with,2022-02-24,Lead Engineer,"Harlem, FL",-1,1,-1,"Everything, you've s job, be happy","Nothing, actually. Everything is fine.",NaN,0.44,"Everything, you've s job, be happyNothing, act...",0.20
3,3,GE,4.0,0,more than 5 years,great experience,2022-02-23,Manager Total Rewards,"Los Angeles, CA",0,2,0,"loved the team, the flexibility to choose spec...",does not offer stock as part of the comp packa...,NaN,0.62,"loved the team, the flexibility to choose spec...",0.84
4,4,GE,4.0,1,more than 5 years,Great place for experience,2022-02-21,Gas Turbine Technician,"Houston, TX",0,0,0,Job security and lots of OT,Raises aren’t good and management expects loyalty,NaN,0.62,Job security and lots of OTRaises aren’t good ...,0.83


In [5]:
print(df.columns)

Index(['Unnamed: 0', 'Company', 'Star_Rating', 'Current_Employee',
       'Work_Duration', 'Review_Title', 'Date_Posted', 'Job_Title',
       'Job_Location', 'Recommend', 'CEO_Approval', 'Business_Outlook', 'Pros',
       'Cons', 'Advice_to_Management', 'Review_Title_sentiment_score_vader',
       'Pros_And_Cons', 'Pros_And_Cons_sentiment_score_vader'],
      dtype='object')


In [6]:
df.drop(['Unnamed: 0', 'Company', 'Review_Title', 'Job_Title', 'Job_Location', 'Pros', 'Cons', 'Advice_to_Management', 'Pros_And_Cons'] , axis=1, inplace=True)

In [7]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Date_Posted,Recommend,CEO_Approval,Business_Outlook,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader
0,5.0,0,NaN,2022-02-25,2,0,1,0.44,-0.08
1,4.0,0,NaN,2022-02-24,0,0,0,0.00,0.44
2,5.0,1,less than 1 year,2022-02-24,-1,1,-1,0.44,0.20
3,4.0,0,more than 5 years,2022-02-23,0,2,0,0.62,0.84
4,4.0,1,more than 5 years,2022-02-21,0,0,0,0.62,0.83


In [9]:
df.dtypes

Star_Rating                            float64
Current_Employee                         int64
Work_Duration                           object
Date_Posted                             object
Recommend                                int64
CEO_Approval                             int64
Business_Outlook                         int64
Review_Title_sentiment_score_vader     float64
Pros_And_Cons_sentiment_score_vader    float64
dtype: object

In [10]:
for i, row in df.iterrows():
	if isinstance(row['Work_Duration'], str):
		if 'less' in row['Work_Duration']:
			years = [int(i) for i in row['Work_Duration'].split() if i.isdigit()]
			df.at[i, 'Work_Duration'] = years[0] - 1
			# row['Work_Duration'] = years[0] - 1
		elif 'more' in row['Work_Duration']:
			years = [int(i) for i in row['Work_Duration'].split() if i.isdigit()]
			df.at[i, 'Work_Duration'] = years[0]
			# row['Work_Duration'] = years[0]

In [11]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Date_Posted,Recommend,CEO_Approval,Business_Outlook,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader
0,5.0,0,NaN,2022-02-25,2,0,1,0.44,-0.08
1,4.0,0,NaN,2022-02-24,0,0,0,0.00,0.44
2,5.0,1,0,2022-02-24,-1,1,-1,0.44,0.20
3,4.0,0,5,2022-02-23,0,2,0,0.62,0.84
4,4.0,1,5,2022-02-21,0,0,0,0.62,0.83


In [12]:
df['Date_Posted'] = pd.to_datetime(df['Date_Posted'], dayfirst=True)

In [20]:
df = df[~(df['Date_Posted'] < '2012-01-01')]

In [21]:
for i in df.columns[df.isnull().any(axis=0)]:     
    df[i].fillna(df[i].mean(),inplace=True)

In [22]:
quarters = pd.Series(pd.date_range('2012-01-01', '2022-12-31', freq='QS'))

In [23]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Date_Posted,Recommend,CEO_Approval,Business_Outlook,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader
0,5.0,0,4.414544,2022-02-25,2,0,1,0.44,-0.08
1,4.0,0,4.414544,2022-02-24,0,0,0,0.00,0.44
2,5.0,1,0.000000,2022-02-24,-1,1,-1,0.44,0.20
3,4.0,0,5.000000,2022-02-23,0,2,0,0.62,0.84
4,4.0,1,5.000000,2022-02-21,0,0,0,0.62,0.83


In [24]:
star_rating_aggregates = dict()
title_senti_aggregates = dict()
pnc_senti_aggregates = dict()
work_duration_aggregates = dict()

current_employee_aggregates = dict()
recommend_aggregates = dict()
ceo_approval_aggregates = dict()
business_outlook_aggregates = dict()


# percentages
recommend_percentages = dict()
ceo_percentages = dict()
business_percentages = dict()
counts = dict()
count = 0

In [25]:
for q in quarters:
	star_rating_aggregates[f'Q{count+1}'] = 0
	title_senti_aggregates[f'Q{count+1}'] = 0
	pnc_senti_aggregates[f'Q{count+1}'] = 0
	work_duration_aggregates[f'Q{count+1}'] = 0

	current_employee_aggregates[f'Q{count+1}'] = []
	recommend_aggregates[f'Q{count+1}'] = []
	ceo_approval_aggregates[f'Q{count+1}'] = []
	business_outlook_aggregates[f'Q{count+1}'] = []

	recommend_percentages[f'Q{count+1}'] = dict()
	ceo_percentages[f'Q{count+1}'] = dict()
	business_percentages[f'Q{count+1}'] = dict()
	for i in range(-1, 3):
		recommend_percentages[f'Q{count+1}'][i] = 0 
		ceo_percentages[f'Q{count+1}'][i] = 0
		business_percentages[f'Q{count+1}'][i] = 0


	counts[f'Q{count+1}'] = 0
	count += 1

In [26]:
for idx, row in df.iterrows():
	for i in range(len(quarters)):
		if i < len(quarters) - 1:
			if row['Date_Posted'] >= quarters[i] and row['Date_Posted'] < quarters[i+1]:
				star_rating_aggregates[f'Q{i+1}'] += row['Star_Rating']
				title_senti_aggregates[f'Q{i+1}'] += row['Review_Title_sentiment_score_vader']
				pnc_senti_aggregates[f'Q{i+1}'] += row['Pros_And_Cons_sentiment_score_vader']
				work_duration_aggregates[f'Q{i+1}'] += row['Work_Duration']

				current_employee_aggregates[f'Q{i+1}'].append(row['Current_Employee'])
				recommend_aggregates[f'Q{i+1}'].append(row['Recommend'])
				ceo_approval_aggregates[f'Q{i+1}'].append(row['CEO_Approval'])
				business_outlook_aggregates[f'Q{i+1}'].append(row['Business_Outlook'])

				recommend_percentages[f'Q{i+1}'][row['Recommend']] += 1
				ceo_percentages[f'Q{i+1}'][row['CEO_Approval']] += 1
				business_percentages[f'Q{i+1}'][row['Business_Outlook']] += 1
				
				counts[f'Q{i+1}'] += 1

In [27]:
import statistics

avg_star_ratings = []
avg_title_senti = []
avg_pnc_senti = []
avg_work_duration = []

avg_current_employee = []
avg_recommend = []
avg_ceo_approval = []
avg_business_outlook = []

# percentages 

recommend_minus_percentages = []
ceo_minus_percentages = []
business_minus_percentages = []

recommend_unknown_percentages = []
ceo_unknown_percentages = []
business_unknown_percentages = []

recommend_same_percentages = []
ceo_same_percentages = []
business_same_percentages = []

recommend_plus_percentages = []
ceo_plus_percentages = []
business_plus_percentages = []


quarts = []

In [28]:

for quarter, value in star_rating_aggregates.items():

	if counts[quarter] != 0:

		avg_star_ratings.append(star_rating_aggregates[quarter] / counts[quarter])
		avg_title_senti.append(title_senti_aggregates[quarter] / counts[quarter])
		avg_pnc_senti.append(pnc_senti_aggregates[quarter] / counts[quarter])
		avg_work_duration.append(work_duration_aggregates[quarter] / counts[quarter])


		avg_current_employee.append(statistics.mode(current_employee_aggregates[quarter]))
		avg_recommend.append(statistics.mode(recommend_aggregates[quarter]))
		avg_ceo_approval.append(statistics.mode(ceo_approval_aggregates[quarter]))
		avg_business_outlook.append(statistics.mode(business_outlook_aggregates[quarter]))

		recommend_minus_percentages.append(recommend_percentages[quarter][-1] / counts[quarter])
		ceo_minus_percentages.append(ceo_percentages[quarter][-1] / counts[quarter])
		business_minus_percentages.append(business_percentages[quarter][-1] / counts[quarter])

		recommend_unknown_percentages.append(recommend_percentages[quarter][0] / counts[quarter])
		ceo_unknown_percentages.append(ceo_percentages[quarter][0] / counts[quarter])
		business_unknown_percentages.append(business_percentages[quarter][0] / counts[quarter])

		recommend_same_percentages.append(recommend_percentages[quarter][1] / counts[quarter])
		ceo_same_percentages.append(ceo_percentages[quarter][1] / counts[quarter])
		business_same_percentages.append(business_percentages[quarter][1] / counts[quarter])

		recommend_plus_percentages.append(recommend_percentages[quarter][2] / counts[quarter])
		ceo_plus_percentages.append(ceo_percentages[quarter][2] / counts[quarter])
		business_plus_percentages.append(business_percentages[quarter][2] / counts[quarter])
		
		quarts.append(quarter)

In [29]:
print(len(avg_star_ratings))

print(len(quarts))

print(len(recommend_minus_percentages))

41
41
41


In [31]:
earnings = [
-1042,
-3843,
1257,
-1131,
-2802,
2636,
-1144,
-1987,
6199,
728,
-9423,
127,
3588,
761,
-22770,
800,
-1147,
-10820,
1360,
1057,
-83,
2336,
2027,
2908,
228,
6301,
2506,
-1360,
-13570,
5152,
3537,
3545,
2999,
3206,
3191,
3133,
3527,
4011,
3491,
3105,
3034
]

earnings = earnings[::-1]
print(len(earnings))

41


In [32]:
print(len(avg_star_ratings))
print(len(avg_pnc_senti))
print(len(avg_work_duration))

print(len(avg_current_employee))
print(len(avg_recommend))
print(len(avg_ceo_approval))
print(len(avg_business_outlook))

print(len(quarts))

41
41
41
41
41
41
41
41


In [33]:
final = pd.DataFrame(
{   'quarters': quarts,
	'ratings': normalize_list(avg_star_ratings),
	'title_senti': normalize_list(avg_title_senti),
	'pnc_senti': normalize_list(avg_pnc_senti),
	'work_duration': normalize_list(avg_work_duration),
	'current_employee': avg_current_employee,
	'recommend': avg_recommend,
	'ceo_approval': avg_ceo_approval,
	'business_outlook': avg_business_outlook,
	'recommend_minus': normalize_list(recommend_minus_percentages),
	'ceo_minus': normalize_list(ceo_minus_percentages),
	'business_minus': normalize_list(business_minus_percentages),
	'recommend_unknown': normalize_list(recommend_unknown_percentages),
	'ceo_unknown': normalize_list(ceo_unknown_percentages),
	'business_unknown': normalize_list(business_unknown_percentages),
	'recommend_same': normalize_list(recommend_same_percentages),
	'ceo_same': normalize_list(ceo_same_percentages),
	'business_same': normalize_list(business_same_percentages),
	'recommend_plus': normalize_list(recommend_plus_percentages),
	'ceo_plus': normalize_list(ceo_plus_percentages),
	'business_plus': normalize_list(business_plus_percentages),
})

In [34]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,...,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus
0,Q1,0.569771,0.954499,0.965373,0.503842,1,2,2,0,0.148303,...,0.000000,0.340773,0.000000,1.000000,None,0.814983,0.000000,0.728025,1.000000,0.000000
1,Q2,0.461594,0.675262,0.942595,0.476743,1,2,2,0,0.270516,...,0.103920,0.402083,0.130598,0.612179,None,0.607133,0.341880,0.564629,0.924384,0.335772
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.472980,...,0.357279,0.064420,0.183721,0.000000,None,0.568201,0.991736,0.762177,0.751299,0.715944
3,Q4,0.266704,0.827978,0.347529,0.373552,1,2,2,1,0.423630,...,0.443284,0.000000,0.170772,0.000150,None,0.278147,1.000000,0.870440,0.945994,0.645312
4,Q5,0.588633,0.833791,0.780301,0.276677,1,2,2,2,0.204858,...,0.211596,0.043089,0.179941,0.039794,None,0.483256,0.867470,1.000000,0.933905,0.849545
5,Q6,0.348993,0.895306,0.590667,0.359017,1,2,2,1,0.523169,...,0.433133,0.105704,0.209387,0.023048,None,0.780053,0.916031,0.677906,0.600114,0.676671
6,Q7,0.633734,0.790989,0.773682,0.528625,1,2,2,2,0.293693,...,0.554699,0.094132,0.241033,0.021460,None,0.675224,0.830022,0.874324,0.766059,0.653755
7,Q8,0.322421,0.498108,0.629464,0.263246,1,2,2,2,0.517955,...,0.412158,0.122789,0.181647,0.060028,None,0.659204,0.828625,0.663875,0.600668,0.694309
8,Q9,0.408722,0.964106,0.701467,0.294502,1,2,2,2,0.316600,...,0.270192,0.311790,0.233444,0.043938,None,0.578993,0.892754,0.623916,0.825595,0.779576
9,Q10,0.548412,1.000000,0.813475,0.214355,1,2,2,2,0.282548,...,0.313916,0.065828,0.238199,0.034472,None,0.491722,0.840941,0.913437,0.820266,0.802510


In [35]:
final = final.dropna(axis = 1)

In [36]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,ceo_same,business_same,recommend_plus,ceo_plus,business_plus
0,Q1,0.569771,0.954499,0.965373,0.503842,1,2,2,0,0.148303,0.463219,0.000000,0.340773,0.000000,1.000000,0.814983,0.000000,0.728025,1.000000,0.000000
1,Q2,0.461594,0.675262,0.942595,0.476743,1,2,2,0,0.270516,0.486264,0.103920,0.402083,0.130598,0.612179,0.607133,0.341880,0.564629,0.924384,0.335772
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.472980,0.780839,0.357279,0.064420,0.183721,0.000000,0.568201,0.991736,0.762177,0.751299,0.715944
3,Q4,0.266704,0.827978,0.347529,0.373552,1,2,2,1,0.423630,0.947365,0.443284,0.000000,0.170772,0.000150,0.278147,1.000000,0.870440,0.945994,0.645312
4,Q5,0.588633,0.833791,0.780301,0.276677,1,2,2,2,0.204858,0.490547,0.211596,0.043089,0.179941,0.039794,0.483256,0.867470,1.000000,0.933905,0.849545
5,Q6,0.348993,0.895306,0.590667,0.359017,1,2,2,1,0.523169,0.605006,0.433133,0.105704,0.209387,0.023048,0.780053,0.916031,0.677906,0.600114,0.676671
6,Q7,0.633734,0.790989,0.773682,0.528625,1,2,2,2,0.293693,0.231376,0.554699,0.094132,0.241033,0.021460,0.675224,0.830022,0.874324,0.766059,0.653755
7,Q8,0.322421,0.498108,0.629464,0.263246,1,2,2,2,0.517955,1.000000,0.412158,0.122789,0.181647,0.060028,0.659204,0.828625,0.663875,0.600668,0.694309
8,Q9,0.408722,0.964106,0.701467,0.294502,1,2,2,2,0.316600,0.316219,0.270192,0.311790,0.233444,0.043938,0.578993,0.892754,0.623916,0.825595,0.779576
9,Q10,0.548412,1.000000,0.813475,0.214355,1,2,2,2,0.282548,0.496960,0.313916,0.065828,0.238199,0.034472,0.491722,0.840941,0.913437,0.820266,0.802510


In [37]:
earnings_df = pd.DataFrame(
	{   'quarters': quarts,
		'earnings': normalize_list(earnings)
	}
)

In [38]:
final.head()

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,ceo_same,business_same,recommend_plus,ceo_plus,business_plus
0,Q1,0.569771,0.954499,0.965373,0.503842,1,2,2,0,0.148303,0.463219,0.000000,0.340773,0.000000,1.000000,0.814983,0.000000,0.728025,1.000000,0.000000
1,Q2,0.461594,0.675262,0.942595,0.476743,1,2,2,0,0.270516,0.486264,0.103920,0.402083,0.130598,0.612179,0.607133,0.341880,0.564629,0.924384,0.335772
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.472980,0.780839,0.357279,0.064420,0.183721,0.000000,0.568201,0.991736,0.762177,0.751299,0.715944
3,Q4,0.266704,0.827978,0.347529,0.373552,1,2,2,1,0.423630,0.947365,0.443284,0.000000,0.170772,0.000150,0.278147,1.000000,0.870440,0.945994,0.645312
4,Q5,0.588633,0.833791,0.780301,0.276677,1,2,2,2,0.204858,0.490547,0.211596,0.043089,0.179941,0.039794,0.483256,0.867470,1.000000,0.933905,0.849545


In [39]:
final.columns

Index(['quarters', 'ratings', 'title_senti', 'pnc_senti', 'work_duration',
       'current_employee', 'recommend', 'ceo_approval', 'business_outlook',
       'recommend_minus', 'ceo_minus', 'business_minus', 'recommend_unknown',
       'ceo_unknown', 'business_unknown', 'ceo_same', 'business_same',
       'recommend_plus', 'ceo_plus', 'business_plus'],
      dtype='object')

In [40]:
final['earnings'] = normalize_list(earnings)

In [41]:
final.dropna(inplace=True)

In [42]:
num_columns = len(final.columns)
print(num_columns)
print(final.columns)

21
Index(['quarters', 'ratings', 'title_senti', 'pnc_senti', 'work_duration',
       'current_employee', 'recommend', 'ceo_approval', 'business_outlook',
       'recommend_minus', 'ceo_minus', 'business_minus', 'recommend_unknown',
       'ceo_unknown', 'business_unknown', 'ceo_same', 'business_same',
       'recommend_plus', 'ceo_plus', 'business_plus', 'earnings'],
      dtype='object')


In [43]:
final.head()

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,...,business_minus,recommend_unknown,ceo_unknown,business_unknown,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,earnings
0,Q1,0.569771,0.954499,0.965373,0.503842,1,2,2,0,0.148303,...,0.000000,0.340773,0.000000,1.000000,0.814983,0.000000,0.728025,1.000000,0.000000,0.887620
1,Q2,0.461594,0.675262,0.942595,0.476743,1,2,2,0,0.270516,...,0.103920,0.402083,0.130598,0.612179,0.607133,0.341880,0.564629,0.924384,0.335772,0.890062
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.472980,...,0.357279,0.064420,0.183721,0.000000,0.568201,0.991736,0.762177,0.751299,0.715944,0.903340
3,Q4,0.266704,0.827978,0.347529,0.373552,1,2,2,1,0.423630,...,0.443284,0.000000,0.170772,0.000150,0.278147,1.000000,0.870440,0.945994,0.645312,0.921227
4,Q5,0.588633,0.833791,0.780301,0.276677,1,2,2,2,0.204858,...,0.211596,0.043089,0.179941,0.039794,0.483256,0.867470,1.000000,0.933905,0.849545,0.904578


In [44]:
final.columns

Index(['quarters', 'ratings', 'title_senti', 'pnc_senti', 'work_duration',
       'current_employee', 'recommend', 'ceo_approval', 'business_outlook',
       'recommend_minus', 'ceo_minus', 'business_minus', 'recommend_unknown',
       'ceo_unknown', 'business_unknown', 'ceo_same', 'business_same',
       'recommend_plus', 'ceo_plus', 'business_plus', 'earnings'],
      dtype='object')

In [45]:
final.to_excel('testing_ge_data.xlsx')